In [1]:
import re
from codebase import bert_predict_class, softmax_and_log_dicts

c:\Users\nilu8\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'bert_predict_class'

Vorhersage testen

In [ ]:
sentence = 'The author of a keygen [MASK] a disassembler to look at the raw assembly code.'
pred = bert_predict_class.bert_predict(sentence)
scores = bert_predict_class.predict_class(pred, softmax_and_log_dicts.LOG_DICTS)
print(pred)
print(scores)

In [62]:
def get_short_class(long_class: str) -> str:
    if long_class == 'Component-Whole(e2,e1)' or long_class == 'Component-Whole(e1,e2)':
        return 'CW'
    if long_class == 'Instrument-Agency(e2,e1)' or long_class == 'Instrument-Agency(e1  ,e2)':
        return 'IA'
    if long_class == 'Member-Collection(e1,e2)' or long_class == 'Member-Collection(e2,e1)':
        return 'MC'
    if long_class == 'Cause-Effect(e2,e1)' or long_class == 'Cause-Effect(e1,e2)':
        return 'CE'
    if long_class == 'Entity-Destination(e2,e1)' or long_class == 'Entity-Destination(e1,e2)':
        return 'ED'
    if long_class == 'Content-Container(e2,e1)' or long_class == 'Content-Container(e1,e2)':
        return 'CC'
    if long_class == 'Message-Topic(e2,e1)' or long_class == 'Message-Topic(e1,e2)':
        return 'MT'
    if long_class == 'Product-Producer(e2,e1)' or long_class == 'Product-Producer(e1,e2)':
        return 'PP'
    if long_class == 'Entity-Origin(e2,e1)' or long_class == 'Entity-Origin(e1,e2)':
        return 'EO'

#### Klassennamen Decoder
Convertiert Klassennamen zu zwei Buchstaben

In [ ]:
with open('test_sentences.txt', 'r') as file:
    lines = file.readlines()

START_AT_SENTENCE_INDEX = 3299

for index, line in enumerate(lines):
    
    # Zeilen mit maskiertem Satz
    if index % 5 == 1:
        # lasse Klasse mit 'other' aus
        sentence = line
        sentence_number = line.split('\t')[0]

    # Zeile mit Label
    if index % 5 == 2:
        y = get_short_class(line.strip())

    # Vorhersage (bei Kommentarzeile)
    if index % 5 == 3:
        # skip Klasse: Other
        if y == None:
            continue
        # skip Satz wenn keine Mask-Token gefunden wurden
        if not re.search(r"\[MASK\]", sentence):
            continue
        # skip bis zum gewünschten Satz
        if int(sentence_number) < START_AT_SENTENCE_INDEX:
            continue
        pred = bert_predict_class.bert_predict(sentence)
        scores = bert_predict_class.predict_class(pred, softmax_and_log_dicts.LOG_DICTS)

        with open('zero_shot_results_LOG.csv', 'a') as results:
            results.write(f"{sentence_number},{y},{scores[0][0]} \n")

Resultate evaluieren

In [12]:
with open('zero_shot_results_LOG_DICTS.csv', 'r') as csv:
    lines = csv.readlines()



    count = 0
    correct_count = 0
    for line in lines:

       #ersetze CT mit MT
        line = line.replace("CT", "MT")

        line = line.split(',')
        count += 1
        line[2] = line[2][:2]
        if line[1] == line[2]:
            correct_count += 1

print(f"Count: {count}")
print(f"Davon korrekt: {correct_count}")
print(correct_count / count)

Count: 4821
Davon korrekt: 1669
0.34619373573947315
